In [ ]:
#This part of code, the goal is to go from mapping and county level weather data to to subgroup data
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path = '/Users/ansonkong/Downloads/rcp85hotter/'
full_historical_data_path = '/Users/ansonkong/Downloads/historic/'

#this just smooth out operation
cutoff_year=2020 #The year prediction dataset start
start_year=2000 #The year historical 
end_year=2100

#Debug mode(only output once)
debug=False


In [1]:
#create population level data
#import require file
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime
from glob import glob



#Path, this will update itself
mapping_path = data_and_output_dir + 'output/merged_rb_control_area_mapping.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path = data_and_output_dir + 'input/county_populations_2000_to_2020.csv'
output_path = data_and_output_dir + 'output/'

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)

#crate looping for interp
years = np.arange(cutoff_year, end_year + 1, 10)
interp_years = np.arange(cutoff_year, end_year + 1)

# Ensure FIPS codes are five zero-padded strings
historical_population_df['county_FIPS'] = historical_population_df['county_FIPS'].astype(str).str.zfill(5)
future_population_df['FIPS'] = future_population_df['FIPS'].astype(str).str.zfill(5)
mapping_df['GEOID'] = mapping_df['GEOID'].astype(str).str.zfill(5)

# Rename 'pop_{year}' columns to just '{year}'
for year in range(start_year, cutoff_year+1):
    if f'pop_{year}' in historical_population_df.columns:
        historical_population_df.rename(columns={f'pop_{year}': str(year)}, inplace=True)



# Assuming 'FIPS' is the column to join on and 'Year' is the column indicating the year in future_population_df
for index, row in future_population_df.iterrows():
    pop_values = [row[str(year)] for year in years if str(year) in row]
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    for year in interp_years:
        future_population_df.at[index, str(year)] = interpolated_values[year - cutoff_year]


# Select relevant columns for historical data
historical_population_df['FIPS']=historical_population_df['county_FIPS']
historical_population_df = historical_population_df[['FIPS'] + [str(year) for year in range(start_year, cutoff_year)]]

# Select relevant columns for future data
future_population_df = future_population_df[['FIPS'] + [str(year) for year in interp_years]]

# Concatenate historical and future dataframes
combined_population_df = pd.concat([historical_population_df, future_population_df]).drop_duplicates(subset=['FIPS']).reset_index(drop=True)
# Merge combined population data with mapping data
# Assuming 'GEOID' in mapping_df corresponds to 'FIPS' in population data
combined_df = pd.merge(mapping_df, combined_population_df, left_on='GEOID', right_on='FIPS', how='inner')
combined_df['FIPS']=combined_df['GEOID']
# Identify FIPS codes in mapping_df that did not successfully join
missing_in_combined = set(mapping_df['GEOID']) - set(combined_df['GEOID'])
print(missing_in_combined)
for column in combined_df.columns:
    print(column)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


set()
STATEFP
COUNTYFP
COUNTYNS
AFFGEOID
GEOID
NAME
LSAD
ALAND
AWATER
geometry
FIPS_x
rb
is_mainland
CNTRL_AREA
FIPS_y
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
FIPS


In [7]:
# main

historical_weather_path = os.path.join(output_path, 'historical_weather')
future_weather_path = os.path.join(output_path, 'future_weather')
os.makedirs(historical_weather_path, exist_ok=True)
os.makedirs(future_weather_path, exist_ok=True)
yearloopbreaker=False
fileloopbreaker=False
rbloopbreaker=False
for year in range(start_year, end_year + 1):

    if yearloopbreaker:break


    print(f'Process start for year: {year}')
    if year <= cutoff_year:
        folder_to_read = os.path.join(full_historical_data_path, str(year))
        folder_to_store = historical_weather_path
    else:
        folder_to_read = os.path.join(full_future_data_path, str(year))
        folder_to_store = future_weather_path
    
    # Initialize a dictionary to hold DataFrames for each rb
    rb_dfs = {}
    # Sort files before processing
    files = sorted(glob(os.path.join(folder_to_read, '*.csv')))
    total_files = len(files)
    processed_files = 0

    # Iterate through each file in the year's folder
    for file_path in files:
        processed_files += 1
        print(f"Processing file {processed_files}/{total_files} ({(processed_files/total_files)*100:.2f}%)")

        if fileloopbreaker:break
        
        meteorology_df = pd.read_csv(file_path)
        if meteorology_df.empty:
            print(f"Skipped empty file: {file_path}")
            continue
        #print(meteorology_df.columns)
        meteorology_df['FIPS'] = meteorology_df['FIPS'].astype(str).str.zfill(5)
        meteorology_df['WSPD'] = np.sqrt(meteorology_df['U10']**2 + meteorology_df['V10']**2)
        
        # Extract date and time from file name
        _, month, day, hour = map(int, os.path.basename(file_path).split('_')[:4])
        utc_time = datetime(year, month, day, hour).strftime('%Y-%m-%d %H:%M:%S')
        meteorology_df['Time_UTC'] = utc_time
        
        # Merge with background data
        merged_df = pd.merge(combined_df, meteorology_df, on='FIPS', how='inner')
        
        # Calculate weighted averages and accumulate in rb_dfs
        merged_df['Population'] = merged_df[str(year)]
        for rb, group in merged_df.groupby('rb'):
            if rbloopbreaker:break
            
            if rb not in rb_dfs:
                rb_dfs[rb] = pd.DataFrame()
            weighted = group.copy()
            
            for col in ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']:
                weighted[col] = weighted[col] * weighted['Population']
            weighted_sum = weighted.groupby('rb').sum()
            for col in ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']:
                weighted_sum[col] = weighted_sum[col] / weighted_sum['Population']
            weighted_sum['Time_UTC'] = utc_time
            weighted_sum=weighted_sum[['Time_UTC','T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']]
            rb_dfs[rb] = pd.concat([rb_dfs[rb], weighted_sum.reset_index()], ignore_index=True)
        if debug:
            fileloopbreaker=True
            yearloopbreaker=True

    
    # Save each RB's accumulated data to its file
    for rb, df in rb_dfs.items():
        output_file = os.path.join(folder_to_store, f"{rb}_WRF_Hourly_Mean_Meteorology_{year}.csv")
        df.sort_values(by='Time_UTC', inplace=True)
        df[['Time_UTC','T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']].to_csv(output_file, index=False)
        print(f"Processed and saved: {output_file} for RB: {rb}")

Process start for year: 2000
Processing file 1/8784 (0.01%)
Processing file 2/8784 (0.02%)
Processing file 3/8784 (0.03%)
Processing file 4/8784 (0.05%)
Processing file 5/8784 (0.06%)
Processing file 6/8784 (0.07%)
Processing file 7/8784 (0.08%)
Processing file 8/8784 (0.09%)
Processing file 9/8784 (0.10%)
Processing file 10/8784 (0.11%)
Processing file 11/8784 (0.13%)
Processing file 12/8784 (0.14%)
Processing file 13/8784 (0.15%)
Processing file 14/8784 (0.16%)
Processing file 15/8784 (0.17%)
Processing file 16/8784 (0.18%)
Processing file 17/8784 (0.19%)
Processing file 18/8784 (0.20%)
Processing file 19/8784 (0.22%)
Processing file 20/8784 (0.23%)
Processing file 21/8784 (0.24%)
Processing file 22/8784 (0.25%)
Processing file 23/8784 (0.26%)
Processing file 24/8784 (0.27%)
Processing file 25/8784 (0.28%)
Processing file 26/8784 (0.30%)
Processing file 27/8784 (0.31%)
Processing file 28/8784 (0.32%)
Processing file 29/8784 (0.33%)
Processing file 30/8784 (0.34%)
Processing file 31/8

In [3]:
file_path

'/Users/ansonkong/Downloads/historic/2010/2010_09_25_08_UTC_County_Mean_Meteorology.csv'

In [5]:
meteorology_df = pd.read_csv(file_path)

In [6]:
meteorology_df

,Unnamed: 0
